In [68]:
using Pkg; Pkg.activate("../.")
using Revise
using QuadrupedBalance
using LinearAlgebra
using Rotations
using SparseArrays
using RobotDynamics
using MeshCat
const QB = QuadrupedBalance

  Activating project at `~/dev/QuadrupedBalance.jl`


QuadrupedBalance

In [170]:
let 
    p = [0.,0.,0.25]
    quat = [1.0, 0.0, 0.0, 0.0]
    J = Matrix(Diagonal([0.05, 0.12, 0.2]))
    m = 12.84
    contacts = [false, true, true, false] # FR, FL, RR, RL
    foot_pos_body = [0.182, -0.1321, -p[3],
                0.182, 0.1321, -p[3], 
                -0.183, -0.132, -p[3], 
                -0.18, 0.132, -p[3]]
    foot_pos_world = copy(foot_pos_body)
    foot_pos_world[3:3:end] .= 0
#     foot_pos_world = [(UnitQuaternion(quat)*foot_pos_body[(i-1)*3+1:(i-1)*3+3] + p) for i in 1:4]
    model = QB.CentroidalPendulum(J, m, foot_pos_world, foot_pos_body, contacts)
    
    q = [p;quat]
    q̇ = zeros(6)
    x = [p..., zeros(3)..., quat..., zeros(3)..., zeros(2)...]
    QB.constraints(model, q)
    a = QB.dcdq(model, q)    

    RobotDynamics.dynamics(model, x, [0,0])
    
    h = 1e-2
    tf = 4.0
    times = 0:h:tf
    xs = zeros(length(times), 15); xs[1,:] = x
    for i in 1:length(times)-1
        if i < 100
            u = [0, 0]
        else 
            u = [50.0, 0.0]
        end
        xs[i+1,:] = QB.dynamics_rk4(model, xs[i,:], u, h)
        q = xs[i+1,[1:3;7:10]]
    end 
    set_mesh!(vis)
    set_animation(vis, model, xs,times, h)
end 

!!


In [60]:
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/chiyen/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8703

In [166]:
function set_mesh!(vis::Visualizer)
    body = MeshCat.HyperRectangle(MeshCat.Vec(-0.1, -0.05, -0.025), MeshCat.Vec(0.2, 0.1, 0.05))
    foot1 = MeshCat.HyperSphere(MeshCat.Point3f0(0), 0.01f0)
    foot2 = MeshCat.HyperSphere(MeshCat.Point3f0(0), 0.01f0)
    setobject!(vis["torso"], body)
    setobject!(vis["foot1"], foot1)
    setobject!(vis["foot2"], foot2)
    settransform!(vis["torso"], MeshCat.Translation(0,0.,0))
end 

function set_animation(vis::Visualizer, model, xs, times, h)
    anim = MeshCat.Animation(convert(Int, floor(1.0 / h)))
    settransform!(vis["foot1"], MeshCat.Translation(model.foot_pos_world[4:6]))
    settransform!(vis["foot2"], MeshCat.Translation(model.foot_pos_world[7:9]))
    for i in 1:length(times)
        p = xs[i,1:3]
        quat = xs[i,7:10]
        
        MeshCat.atframe(anim, i) do 
            H = MeshCat.compose(MeshCat.Translation(p), MeshCat.LinearMap(UnitQuaternion(quat)))
            settransform!(vis["torso"], H)
        end
    end
    
    MeshCat.setanimation!(vis, anim)
    println("!!")
end 


set_animation (generic function with 4 methods)

In [61]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))